In [ ]:
#pull up data in units of distance/time
#our conversion = (pixels/frame)(36000um/1228pixels)(1frame/5min) = 5.86

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df1 = pd.read_excel('Velocity_Map_X.xlsx',header=None)
df2 = pd.read_excel('Velocity_Map_Y.xlsx',header=None)

vxt = (df1.values*5.86)
vyt = (df2.values*5.86)

In [ ]:
#compute velocity magnitude
vmag = np.sqrt(vxt**2 + vyt**2)

#save velocity magnitude map as an Excel file
vmag_df = pd.DataFrame(vmag)
vmag_df = vmag_df.iloc[::-1].reset_index(drop=True) # inverting the rows so that it matches the image
output_filename = "VMag_Map.xlsx"
vmag_df.to_excel(output_filename, index=False, header=False)

In [ ]:
#upsampling velocity to match size of tif images

from skimage.transform import resize
import seaborn as sns

#convert dataframe to numpy array
image_data = vmag

#upsample to match size of stereoscope image
upsampled_data = resize(image_data, (920, 1228), mode='reflect', anti_aliasing=True)

#convert back to dataframe
df_upsampled = pd.DataFrame(upsampled_data[::-1])
vmag_df = df_upsampled

#plot velocity data as a heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(vmag_df, annot=False, cmap='viridis')
plt.show()

In [ ]:
#importing tif image for area selection

from skimage.color import rgb2gray

image_array = plt.imread('Image.tif')
grayscale_array = rgb2gray(image_array)
df = pd.DataFrame(grayscale_array)
result_data_df = df

In [ ]:
#polygon selection of only areas exhibiting growth in frame of interest

%matplotlib qt
from matplotlib.widgets import LassoSelector, Button
from matplotlib.path import Path

selected_polygons = []

def onselect(verts):
    selected_polygons.append(verts)
    polygon = plt.Polygon(verts, fill=False, edgecolor='red', alpha=0.5)
    ax.add_patch(polygon)

#create the plot
fig, ax = plt.subplots()
ax.matshow(result_data_df, cmap='viridis')

#button to apply changes after all selections
def apply_changes(event):
    global result_data_df
    
    #initialize a mask of the same shape as the dataframe with all True values
    mask = np.ones(result_data_df.shape, dtype=bool)
    
    for verts in selected_polygons:
        path = Path(verts)
        for r in range(result_data_df.shape[0]):
            for c in range(result_data_df.shape[1]):
                if path.contains_point((c, r)):  # if inside the polygon
                    mask[r, c] = False

    #set values outside the polygons to NaN using the mask
    result_data_df[mask] = np.nan
    plt.close()  #close the interactive plot after applying selections

ax_button = plt.axes([0.7, 0.05, 0.2, 0.075])
btn = Button(ax_button, 'Apply Selections')
btn.on_clicked(apply_changes)

# Use the LassoSelector widget
lasso_selector = LassoSelector(ax, onselect)
plt.show()

#print the updated vmag_df
print(result_data_df)

In [ ]:
#saving selected area dataframe as an excel file
#only use this cell to save selected data, if data is already saved proceed to next cell

%matplotlib inline

result_data_df.to_excel("area_selection.xlsx", index=False)

In [ ]:
#applying saved result_data_df to tif image
#only use this cell once you have saved your area selection and need to recall the data

%matplotlib inline

#load the data from the Excel file into a Pandas DataFrame
result_data_df = pd.read_excel("area_selection.xlsx")

#set the corresponding data points in vmag_df as NaN where result_data_df has NaN
vmag_df = vmag_df.where(result_data_df.notna())
vmag_df = vmag_df.replace(0, np.nan)

#show the plot
plt.figure(figsize=(10, 7))
sns.heatmap(vmag_df, annot=False, cmap='viridis')
plt.show()

In [ ]:
#set the corresponding data points in result_data_df as NaN where vmag_df has NaN
vmag_df = vmag_df.where(result_data_df.notna())
vmag_df_nan = vmag_df.replace(0, np.nan)

#show the plot
plt.figure(figsize=(10, 7))  # adjust the figure size as needed
sns.heatmap(vmag_df, annot=False, cmap='viridis')  # annot=True will annotate each cell with its value
plt.show()

In [ ]:
#process and flatten corresponding data points in each dataframe for ease of plotting 

#replace placeholder zeroes with NaN so they are not read as your data
vmag_df.replace(0, np.nan, inplace=True)
result_data_df[vmag_df.isna()] = np.nan

#convert DataFrames to NumPy arrays
intensity_data = result_data_df.values.flatten()
vmag_data = vmag_df.values.flatten()

import math

intensity_data = [item for item in intensity_data if not math.isnan(item)]
vmag_data = [item for item in vmag_data if not math.isnan(item)]

In [ ]:
#save files for a combined plot

#create dataFrames from your data
intensity_df = pd.DataFrame({'Intensity': intensity_data})
vmag_df = pd.DataFrame({'Vmag': vmag_data})

#define the file names
intensity_file = "intensity_data.xlsx"
vmag_file = "vmag_data.xlsx"

#save the dataFrames as Excel files
intensity_df.to_excel(intensity_file, index=False)
vmag_df.to_excel(vmag_file, index=False)

In [ ]:
#generate combined velocity CDF of selected regions only, for all time points

#file names and corresponding times
file_names = ['vmag_data_1.xlsx', 'vmag_data_2.xlsx', 'vmag_data_3.xlsx', 'vmag_data_4.xlsx']
times = [timepoint1, timepoint2, timepoint3, timepoint4]  #we used hours, but this can be any unit of time

#create a figure
fig, ax = plt.subplots()

#iterate over each file for combined plot
for file_name, time in zip(file_names, times):
    #load the data from Excel file
    df = pd.read_excel(file_name)
    
    #sxtract velocity data
    velocity_data = df['Vmag'].values
    
    #sort the data
    sorted_velocity = np.sort(velocity_data)
    
    #calculate the cumulative probability
    cumulative_prob = np.linspace(0, 1, len(sorted_velocity))
    
    #plot CDF for velocity with labels
    ax.plot(sorted_velocity, cumulative_prob, marker='o', label=f'{time} hr', markersize=1)

#set labels and title
ax.set_xlabel('Velocity Magnitude')
ax.set_ylabel('Cumulative Probability')
ax.set_title('CDF Velocity')

#add legend
ax.legend()

#disable grid
ax.grid(False)

#show plot
plt.show()